# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.

%load_ext dotenv
%dotenv 

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:

# Write your code below.
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA") 

# Recursive search for parquet files
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
print("First 5 parquet files:", parquet_files[:5])  # preview first 5
#dd_px = dd.read_parquet(parquet_files).set_index("ticker")

First 5 parquet files: ['../../05_src/data/prices/MTN/MTN_2005/part.0.parquet', '../../05_src/data/prices/MTN/MTN_2005/part.1.parquet', '../../05_src/data/prices/MTN/MTN_2009/part.0.parquet', '../../05_src/data/prices/MTN/MTN_2009/part.1.parquet', '../../05_src/data/prices/MTN/MTN_2004/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

import dask.dataframe as dd

# Read into Dask DataFrame
ddf = dd.read_parquet(parquet_files)

# Feature engineering per ticker
dd_feat = (
    ddf
    .groupby("ticker", group_keys=False)
    .apply(lambda df: df.assign(
        Close_lag_1 = df["Close"].shift(1),
        Adj_Close_lag_1 = df["Adj Close"].shift(1),
        returns = (df["Close"] / df["Close"].shift(1)) - 1,
        hi_lo_range = df["High"] - df["Low"]
    ))
)



/tmp/ipykernel_84238/1973921302.py:12: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  .apply(lambda df: df.assign(
/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.

#Convert dask dataframe to pandas dataframe
df_feat = dd_feat.compute()

df_feat["returns_ma_10"] = (
    df_feat
    .groupby("ticker")["returns"]
    .transform(lambda x: x.rolling(window=10).mean())
)



/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:122: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
/usr/local/python/3.12.1/lib/python3.12/site-packages/dask/dataframe/groupby.py:12

In [6]:
df_feat.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
184612,1993-01-04,5.9375,5.9375,5.8750,5.8750,0.660554,4200.0,UBP.csv,UBP,1993,NaN,NaN,NaN,0.0625,NaN
184613,1993-01-05,5.8750,6.0000,5.8750,6.0000,0.674608,2000.0,UBP.csv,UBP,1993,5.8750,0.660554,0.021277,0.1250,NaN
184614,1993-01-06,5.9375,5.9375,5.9375,5.9375,0.667581,800.0,UBP.csv,UBP,1993,6.0000,0.674608,-0.010417,0.0000,NaN
184615,1993-01-07,5.9375,5.9375,5.8750,5.8750,0.660554,9200.0,UBP.csv,UBP,1993,5.9375,0.667581,-0.010526,0.0625,NaN
184616,1993-01-08,5.8750,5.8750,5.8750,5.8750,0.660554,3200.0,UBP.csv,UBP,1993,5.8750,0.660554,0.000000,0.0000,NaN
184617,1993-01-11,6.0000,6.0000,5.9375,6.0000,0.674608,23200.0,UBP.csv,UBP,1993,5.8750,0.660554,0.021277,0.0625,NaN
184618,1993-01-12,6.0625,6.0625,6.0000,6.0625,0.681635,5200.0,UBP.csv,UBP,1993,6.0000,0.674608,0.010417,0.0625,NaN
184619,1993-01-13,6.0625,6.0625,6.0000,6.0000,0.674608,5600.0,UBP.csv,UBP,1993,6.0625,0.681635,-0.010309,0.0625,NaN
184620,1993-01-14,5.9375,6.0000,5.9375,6.0000,0.674608,11200.0,UBP.csv,UBP,1993,6.0000,0.674608,0.000000,0.0625,NaN
184621,1993-01-15,6.0000,6.0625,5.9375,6.0625,0.681635,13600.0,UBP.csv,UBP,1993,6.0000,0.674608,0.010417,0.1250,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Not quite sure as I am still trying to understand everything. However, Dask is better to use when we have very large or complex datasets. It helps with processing as large datasets ordinarily will use lots of memory and take a longer time to process. The converse is true for Pandas. It is better for small datasets. You are able to get the results needed more quickly and it is doesn't drain memory.

It does seem like it would have been better to calculte it in dask. I do see performance issues in terms of speed with my computer doing the display above.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.